# 1. Get the 15 variables from this raster for all Peru departments polygons

## 1.1 Install and import

In [25]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q

Note: you may need to restart the kernel to use updated packages.


In [27]:
!pip install ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/Users/isaisilva/anaconda3/envs/geowombat/lib/python3.8/site-packages/jupyterlab/debuglog.py:56: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/Users/isaisilva/anaconda3/envs/geowombat/lib/python3.8/site-packages/jupyterlab/debuglog.py:57: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/Users/isaisilva/anaconda3/envs/geowombat/lib/python3.8/site-packages/jupyterlab/debuglog.py:58: UserWarning: See the log file for details: /var/fold

In [34]:
pip install rasterstats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [35]:
import geowombat as gw

In [36]:
import geopandas as gpd
from rasterstats import zonal_stats
import pandas as pd

In [37]:
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import mapping
import rasterio
import geopandas as gpd
import rasterio

import matplotlib.pyplot as plt
from rasterio.plot import show

import os
from rasterio.mask import mask

# 1.2

In [39]:
import geopandas as gpd

# Define the relative path to the shapefile from the notebook location
shapefile_path = '../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp'

# Read the shapefile using Geopandas
departments = gpd.read_file(shapefile_path)

# Transformar el CRS de los departamentos al CRS del raster si es necesario
# Asumiendo que los rasters están en 'esri:54009'
departments = departments.to_crs('esri:54009')


print(departments.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [40]:
import rasterio
from rasterio.merge import merge
import rasterio
from rasterio.mask import mask
import pandas as pd

In [41]:
# Rutas a los archivos raster
raster_files = ['tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif',
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif', 
                'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif']

In [42]:
# Inicializar lista para almacenar los resultados de cada raster
all_stats = []

In [44]:
# Iterar sobre cada archivo raster
for raster_path in raster_files:
    # Calcular estadísticas zonales
    stats = zonal_stats(departments, raster_path, stats="count sum", categorical=True, all_touched=True)
    # Convertir estadísticas a DataFrame
    stats_df = pd.DataFrame(stats)
    # Concatenar con la geometría y la información de los departamentos
    df = pd.concat([departments.reset_index(drop=True), stats_df], axis=1)
    all_stats.append(df)

RasterioIOError: 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif' not recognized as a supported file format.

In [ ]:
# Combinar resultados si hay múltiples DataFrames
# Suponiendo que 'count' representa el número de píxeles de MSZ por departamento
# final_stats = pd.concat(all_stats).groupby('NOMBRE_DEPARTAMENTO').sum().reset_index()

In [ ]:
#Especificar Variables de MSZ:  Identificar qué valores en el raster representan las "15 variables" mencionadas. 
#Esto puede referirse a diferentes tipos de cobertura de MSZ identificados por valores únicos en el raster. 

# Calcular el Porcentaje de Cobertura: Para cada departamento, calcular el porcentaje del área cubierta por cada una de las variables de MSZ. 
# Esto implica dividir el área cubierta por cada valor de MSZ por el área total del departamento y multiplicar por 100.


# Se tiene un DataFrame 'df' resultante de 'zonal_stats'

# Calcular el área total del departamento en metros cuadrados (o la unidad de área correspondiente en 'esri:54009')
# puede requerir una conversión adicional dependiendo del CRS de 'departments'

# Para cada valor de MSZ, calcular el porcentaje de cobertura. Esto podría implicar iterar sobre los valores únicos
# en el raster y calcular el área cubierta por cada uno, dividiendo por el área total del departamento.
# La implementación exacta dependerá de cómo 'zonal_stats' devuelve los conteos y sumas para cada categoría (valor de MSZ).
